In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

distributed_cache_file = 'stop_words_en.txt'
def read_stop_words(file_path):
    return set(word.strip().lower() for word in open(file_path))
stop_words = read_stop_words(distributed_cache_file)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:myGroup,Total_words,%d" % 1
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:myGroup,Stop_words,%d" % 1
#         print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
%%writefile myScript.py

import sys
reload(sys)

nums = []
for line in sys.stdin:
    try:
        num = line.strip()
        num = float(num)
#         num = float(num)
    except ValueError as e:
        continue
    
    nums.append(num)
print nums[0]*100.0 / nums[1]

Overwriting myScript.py


In [3]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=0

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -numReduceTasks 0 \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> output.log

# print the answer
cat output.log | egrep "*_words" | grep -Eo [0-9]+ | python2 myScript.py

# print log to stderr for grader
cat output.log >&2

38.4403690091


rm: `wordcount_result_1516595458685797': No such file or directory
18/01/22 04:31:02 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/22 04:31:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/22 04:31:04 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/22 04:31:04 INFO mapreduce.JobSubmitter: number of splits:2
18/01/22 04:31:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1516574726089_0023
18/01/22 04:31:04 INFO impl.YarnClientImpl: Submitted application application_1516574726089_0023
18/01/22 04:31:04 INFO mapreduce.Job: The url to track the job: http://7924f258b69f:8088/proxy/application_1516574726089_0023/
18/01/22 04:31:04 INFO mapreduce.Job: Running job: job_1516574726089_0023
18/01/22 04:31:11 INFO mapreduce.Job: Job job_1516574726089_0023 running in uber mode : false
18/01/22 04:31:11 INFO mapreduce.Job:  map 0% reduce 0%
18/01/22 04:31:28 INFO mapreduce.Job:  map 60% reduce 0%
18/01/22 04:3